In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

plt.rcParams['font.family'] = 'Arial'
plt.rcParams['font.size'] = 7
plt.rcParams['lines.linewidth'] = 2

%matplotlib inline

In [ ]:
library_df =pd.read_excel('../data_tables/crispey3_libraries_by_pool_number.xlsx')

In [ ]:
counts = pd.read_csv('../data_tables/mapped_counts_pbc_umi_CAFF_COCL_NACL_LOV.tsv', '\t')
counts['barcode_id'] = counts.oligo_name.str.split('_').str[1:3].str.join('_')
counts['well'] = counts.barcode_id.str.split('_').str[0].astype(float)
counts = counts[counts['well'].isin(library_df['ergosterol'].dropna().tolist())]
for col in counts.filter(regex = 'counts').columns:
    counts[col] = (counts[col]/counts[col].sum())*1000000

In [ ]:
fig, ax = plt.subplots(figsize = (2.1,2.0))

var = ['018_053']
condition = 'NACL'

hit = counts[counts['barcode_id'].isin(var)].filter(regex = condition+'|barcode_id')
gens = [0,4.2900394343,8.4380959921,12.59468422,16.5642368546,21.0984075671]
cmap = plt.get_cmap("tab10")
for i,barcode in enumerate(var):
    bc = hit[hit['barcode_id']==barcode].filter(regex = condition).T
    bc['tp'] = bc.index.str.split('_').str[0:3].str.join('_')
    bc = bc.groupby('tp').agg('sum')
    
    for col in bc.columns:
        color = 'lightgrey'
        plt.plot(gens, (bc.loc[:, col])/(bc.loc[:, col].values[0]), c= 'lightgrey', alpha = 1)
hit[hit['barcode_id']==var[0]].filter(regex = condition).T
plt.plot(gens, (bc.loc[:, 12657])/(bc.loc[:, 12657].values[0]), c= 'chartreuse', alpha = 1)

fixed_stacked = hit.drop(12657).filter(regex='counts').T

fixed_stacked['tp'] = fixed_stacked.index.str.split('_').str[0:3].str.join('_')

fixed_stacked = fixed_stacked.groupby('tp').agg('sum')
sumstack = fixed_stacked.sum(axis = 1)
f = sumstack/(sumstack['P1_NACL_T1'])

plt.plot(gens, f, c = 'black', linestyle = 'dotted')
stacked = hit.filter(regex = 'counts').T
stacked['tp'] = stacked.index.str.split('_').str[0:3].str.join('_')

stacked = stacked.groupby('tp').agg('sum')
sumfullstack = stacked.sum(axis = 1)
f2 = sumfullstack/(sumfullstack['P1_NACL_T1'])

plt.plot(gens, f2, c = 'chartreuse', linestyle = 'dotted')    
custom_lines = [Line2D([0], [0], color='chartreuse', lw=2),
                Line2D([0], [0], color='chartreuse', lw=2, linestyle = 'dotted'),
                Line2D([0], [0], color='grey', lw=2),
               Line2D([0], [0], color='black', lw=2, linestyle = 'dotted')]
plt.yscale('log')
plt.xticks([0,5,10,15,20,25]);
plt.xlim(0,22)
plt.xticks(fontsize =7)
plt.yticks(fontsize =7)
plt.xlabel('Timepoint', fontsize = 7)
plt.ylabel('Change in CPM from T1', fontsize = 7)
plt.legend(handles = custom_lines, labels = ['Outlier UMI', 'Sum w/ Outlier', 'Remaining UMIs', 'Sum w/o Outlier'], fontsize =6 )
plt.savefig('../GxE_Figures/Figure_1/figure1h.svg')